# Example: Half Adder
In this example we write a quantum algorithm to aperform an add operation between two qubits and store the result in two qubits that act as output register.

The truth table of the circuit is
| in q1 | in q2 | out q4 | out q3 | out  (integer) |
|-------|-------|--------|--------|----------------|
| 0     | 0     | 0      | 0      | 0              |
| 0     | 1     | 0      | 1      | 1              |
| 1     | 0     | 0      | 1      | 1              |
| 1     | 1     | 1      | 0      | 2              |

The circuit that we will code to implements a half adder (with an additional unimportant phase) is as follows

![half adder](half_adder.png)

The output of q4 and q3 can be interpreted as the output integer as `2^1*q4+2^0*q3`.

The to `X` operations at the beginning are used to generate input, in this case both input qubits are set to `1`.

In [27]:
# A small helper routine for nice output
def get_binary(x: int, size: int) -> str:
    """Return a string representation of int x with size positions."""
    return format(x, 'b').zfill(size)

## Input initialization

Requried is the first step of the circuit: Flip the input qubits from `0 ` to `1`. We use the qoqo toolkit to represent quantum circuits.

A `Circuit` is the main class to represent quantum circuits. The operations module contains one- and two-qubit operations such as Hadamard, PauliX or CNOT.

The initialization circuit contains two gates, `PauliX` on the first two qubits.

Write code that imports required tools from qoqo and write a circuit that flips the two input qubits into `1`.
Hint: Use the PauliX operation

In [28]:
# Code here
from qoqo import Circuit
from qoqo import operations as ops

init = Circuit()
init += ops.PauliX(0)
init += ops.PauliX(1)

## Verify the input
To verify that the qubits have been flipped we test it on a (simulated) quantum computer. We use the `qoqo_quest` library to interface to quest and run the simulation.

We require two additional operations,

* `DefinitionBit` - Create a classical bit register to store measured bit values.
* `MeasureQubit` -  Measure a qubit and store the input in the classical bit register.

A circuit can be simulated on the backend using `run_circuit`.

In [29]:
# Code goes here
from qoqo_quest import Backend
measure = Circuit()
measure += ops.DefinitionBit('ro', 2, is_output=True)
measure += ops.MeasureQubit(0, 'ro', 0)
measure += ops.MeasureQubit(1, 'ro', 1)
(res_bits, _, _) = Backend(4).run_circuit(init + measure)
print(res_bits)

{'ro': [[True, True]]}


## Circuit to set first output qubit
The next part of the circuit that we implement changes the first output qubit according to the inputs. The truth table for this is


The truth table of this operation is

| in q1 | in q2 |  out q3 |
|-------|-------|---------|
| 0     | 0     |  0      |
| 0     | 1     |  1      |
| 1     | 0     |  1      |
| 1     | 1     |  0      |

For this we require the `CNOT` operation. We need one CNOT with `control=0` and one with `control=1`. The target for both operations is qubit `2`.

Hint: Don't forget to remove the measurement operations from the verification step!

In [30]:
# Code goes here
from qoqo import Circuit
from qoqo import operations as ops

circuit = Circuit()
circuit += ops.CNOT(control=0, target=2)
circuit += ops.CNOT(control=0, target=1)

Verify the correct seting of qubit 2. It could be usefull to comment out one of the `PailiX` gates in the initialization to cahnge the measured value from 0 to 1.

In [31]:
# Code goes here
from qoqo_quest import Backend
measure = Circuit()
measure += ops.DefinitionBit('ro', 1, is_output=True)
measure += ops.MeasureQubit(2, 'ro', 0)
(res_bits, _, _) = Backend(4).run_circuit(init + circuit + measure)
print(res_bits)

{'ro': [[True]]}


## Set the second output qubit
Finally, we need to set the second output representing the most relevant bit (2^1). The truth table of this operation is

| in q1 | in q2 |  out q4 |
|-------|-------|---------|
| 0     | 0     |  0      |
| 0     | 1     |  0      |
| 1     | 0     |  0      |
| 1     | 1     |  1      |

We see that q4 is flipped only if both inputs are 1. This corresponds to a CCNOT or Toffoli gate. In the circuit diagram at the beginning the Tofolli gate has been decomposed into 1- and 2-qubit gates.

For this step we require the `CNOT` operation, the `Hadamard` operation and a `Rotatey(\pm\pi/4)` operation. Implement the last step and validate the output.

Hint: Don't forget to remove the measurement steps from verification.

In [32]:
import math

circuit += ops.CNOT(control=0, target=2)
circuit += ops.CNOT(control=0, target=1)

circuit += ops.RotateY(3, math.pi / 4)
circuit += ops.CNOT(0, 3)
circuit += ops.RotateY(3, -math.pi / 4)
# Controlled Z
circuit += ops.Hadamard(3)
circuit += ops.CNOT(1, 3)
circuit += ops.Hadamard(3)
# Controlled H
circuit += ops.RotateY(3, math.pi / 4)
circuit += ops.CNOT(0, 3)
circuit += ops.RotateY(3, -math.pi / 4)

measure = Circuit()
measure += ops.DefinitionBit('ro', 2, is_output=True)
measure += ops.MeasureQubit(2, 'ro', 0)
measure += ops.MeasureQubit(3, 'ro', 1)
(res_bits, _, _) = Backend(4).run_circuit(init + circuit + measure)
print(res_bits)

{'ro': [[False, True]]}


## Further ideas:
- replace the X gates in the initializaiton step by `Hadamard` gates
- repeat the measurement for ~1000 times and plot a histogram of the outputs. Do you understand them?
